In [1]:
import matplotlib.pyplot as plt  # matplotlib version: 3.6.3
import numpy as np
import pandas as pd
import scipy.fft as fft
import plotly.graph_objects as go
import plotly.express as px
import webbrowser
from scipy.optimize import curve_fit
import os
import re
import sympy as sp

In [2]:
opt_parameters = [   34.20949562,  1317.99024017,  -156.27590486,  -544.54377573,
       -5732.4692288 ,   282.56780882,  9396.56391293,  -148.9753875 ,
         -51.41313683,  1244.93126392]

In [3]:
def aggIn(KP, KD, args):
    if len(args) < 10:
        raise ValueError("Wrong number of arguments")
    
    return args[0] + args[1]*KP + args[2]*KD + args[3]*KP*KD + args[4]*KP**2 + args[5]*KD**2 + args[6]*KP**3 + args[7]*KD**3 + args[8]*KP*KD**2 + args[9]*KP**2*KD

In [4]:
z = 110
KP = 0.15
KD = sp.Symbol('KD')

eq = sp.Eq(aggIn(KP, KD, opt_parameters), z)
sol = sp.solve(eq, KD)

print([s for s in sol if s.is_real and s >= 0.001 and s <= 1])


[0.141603536646224]


In [35]:
def PIDgen(z):
    if z < 60 or z > 160:
        raise ValueError("AggressivenesKD index out of range")
    
    z = float(z)
    k = 10    
    KP_A = 0.001
    KP_B = 0.3
    
    KD_A = 0.001
    KD_B = 1
    res_list = []
    
    while (len(res_list) < k):
        
        # TODO: make the center of the gaussian bell closer to a desired KP (especially for high values of z)
        KP = 0
        while(KP < KP_A or KP > KP_B):
            KP = np.random.normal((KP_B - KP_A) / 2, (KP_B - KP_A) / 4)
        
        KD = sp.Symbol('KD')
        eq = sp.Eq(aggIn(KP, KD, opt_parameters), z)
        sol = sp.solve(eq, KD)
        sol = [s for s in sol if s.is_real and s >= KD_A and s <= KD_B]
        
        for s in sol:
            res_list.append({'KP' : KP, 'KI' : 0.01, 'KD' : s}) 
        
    return res_list

In [68]:
res = PIDgen(140)

In [69]:
for r in res:
    r['AggIn'] = aggIn(KP=r['KP'], KD=r['KD'], args=opt_parameters)
    print(r, end='')
    print(' -> AggIn=', r['AggIn'])

{'KP': 0.29262885687573104, 'KI': 0.01, 'KD': 0.140311150626445, 'AggIn': 140.000000000000} -> AggIn= 140.000000000000
{'KP': 0.20470752295693062, 'KI': 0.01, 'KD': 0.0209506973358182, 'AggIn': 140.000000000000} -> AggIn= 140.000000000000
{'KP': 0.25335801892287063, 'KI': 0.01, 'KD': 0.0658184392293287, 'AggIn': 140.000000000000} -> AggIn= 140.000000000000
{'KP': 0.20298673083315424, 'KI': 0.01, 'KD': 0.0196768653157456, 'AggIn': 140.000000000000} -> AggIn= 140.000000000000
{'KP': 0.1870539497885405, 'KI': 0.01, 'KD': 0.00785922875174082, 'AggIn': 140.000000000000} -> AggIn= 140.000000000000
{'KP': 0.24606513369138067, 'KI': 0.01, 'KD': 0.0571556566451075, 'AggIn': 140.000000000000} -> AggIn= 140.000000000000
{'KP': 0.23617473830089009, 'KI': 0.01, 'KD': 0.0468869135122278, 'AggIn': 140.000000000000} -> AggIn= 140.000000000000
{'KP': 0.266132983237502, 'KI': 0.01, 'KD': 0.0839531941417359, 'AggIn': 140.000000000000} -> AggIn= 140.000000000000
{'KP': 0.24149970278551292, 'KI': 0.01, 'KD

In [38]:
def esd(yt, fs):
    # computes ESD of a signal yt, keeping only positive frequencies! -> NOTE: this implies that total energy is halved
    N = len(yt)
    f = fft.rfftfreq(N, 1/fs)
    Sxx = np.square(1 / fs) *  np.square(np.abs(fft.rfft(yt)))
    
    return f, Sxx

In [39]:
dt = 0.005
fs_FTP = 10
k_subsample = int(1 / (dt * fs_FTP))

In [45]:
KPs = []
KI = 0.01
KDs = []

pattern_file = r'PID_([\d\.]+)_([\d\.]+)_([\d\.]+).csv'
pattern_folder = r'KP=([\d\.]+)'

points = []

z = []

for folder in os.listdir('C:/Users/matte/Documents/NL3/CustomCarla/carla/PythonAPI/AggressiveDrivingSignals/parameters_exploration/Kx_exploration_data'):
    match_folder = re.match(pattern_folder, folder)
    KP = float(match_folder.group(1))
    KPs.append(KP)
    
    same_KP = []

    for file in os.listdir(os.path.join('C:/Users/matte/Documents/NL3/CustomCarla/carla/PythonAPI/AggressiveDrivingSignals/parameters_exploration/Kx_exploration_data', folder)):
        match_file = re.match(pattern_file, file)
        if float(match_file.group(2)) == KI:
            KD = float(match_file.group(3))
                        
            filename = 'C:/Users/matte/Documents/NL3/CustomCarla/carla/PythonAPI/AggressiveDrivingSignals/parameters_exploration/Kx_exploration_data' + os.sep + folder + os.sep + file
            signals_df = pd.read_csv(filename, sep=',', header=0)
            x_t = np.array(signals_df['Time'], dtype=float)
            y_rv = np.array(signals_df['Velocity'], dtype=float)
            y_tv = np.array(signals_df['Target Velocity'], dtype=float)
            y_tp = np.array(signals_df['Throttle'], dtype=float)

            x_t = x_t[::k_subsample]
            y_rv = y_rv[::k_subsample]
            y_tv = y_tv[::k_subsample]
            y_tp = y_tp[::k_subsample]
            
            f, Sxx = esd(y_tp, fs_FTP)

            AggIn = float(np.mean(Sxx) / np.std(y_tv - y_rv))

            # append KD value to KDs list, if the element is not already present
            if KD not in KDs:
                KDs.append(KD)  
            
            same_KP.append(AggIn)
            
            points.append({'KP': KP, 'KI' : KI, 'KD': KD, 'AggIn' : AggIn})
    
    z.append(same_KP)


        

In [70]:
res

[{'KP': 0.29262885687573104,
  'KI': 0.01,
  'KD': 0.140311150626445,
  'AggIn': 140.000000000000},
 {'KP': 0.20470752295693062,
  'KI': 0.01,
  'KD': 0.0209506973358182,
  'AggIn': 140.000000000000},
 {'KP': 0.25335801892287063,
  'KI': 0.01,
  'KD': 0.0658184392293287,
  'AggIn': 140.000000000000},
 {'KP': 0.20298673083315424,
  'KI': 0.01,
  'KD': 0.0196768653157456,
  'AggIn': 140.000000000000},
 {'KP': 0.1870539497885405,
  'KI': 0.01,
  'KD': 0.00785922875174082,
  'AggIn': 140.000000000000},
 {'KP': 0.24606513369138067,
  'KI': 0.01,
  'KD': 0.0571556566451075,
  'AggIn': 140.000000000000},
 {'KP': 0.23617473830089009,
  'KI': 0.01,
  'KD': 0.0468869135122278,
  'AggIn': 140.000000000000},
 {'KP': 0.266132983237502,
  'KI': 0.01,
  'KD': 0.0839531941417359,
  'AggIn': 140.000000000000},
 {'KP': 0.24149970278551292,
  'KI': 0.01,
  'KD': 0.0522279560708200,
  'AggIn': 140.000000000000},
 {'KP': 0.24974979621118165,
  'KI': 0.01,
  'KD': 0.0614010755640671,
  'AggIn': 140.00000000

In [71]:
plot_folder = 'C:/Users/matte/Documents/NL3/CustomCarla/carla/PythonAPI/AggressiveDrivingSignals/parameters_exploration/Kx_exploration_plots'

real_data = go.Surface(x = KPs, y = KDs, z = np.array(z).transpose(), colorscale = 'Viridis', name='Real data', opacity=0.8)
pred_data = go.Scatter3d(x = [float(d['KP']) for d in res], y = [float(d['KD']) for d in res], z = [float(d['AggIn']) for d in res], 
                         mode = 'markers', marker = dict(size = 5, opacity = 0.8, color = 'red'), name='Predicted data')
fig = go.Figure(data = [real_data, pred_data])
fig.update_layout(  title = 'AggIn with fixed KI = 0.01',
                    scene=dict(xaxis_title='KP', yaxis_title='KD', zaxis_title='AggIn'))

fig.write_html(plot_folder + os.sep + 'real_vs_pred_2.html') 
webbrowser.open(plot_folder + os.sep + 'real_vs_pred_2.html',new=2)

True